In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt        
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression
import numpy as np
import os 

In [3]:
def calculate_mean(df,column_name):
   return df[column_name].mean()

def replace_mis_val_mean(df,column_name):
    mean_val = calculate_mean(df,column_name)
    df.loc[df[column_name].isna(),column_name] = mean_val
    return df 
    


    
def mis_replace_linear_regression(df,X_col=["minPrice"],Y_col="modalPrice"):
    #Missing value Treatment 
    #Method 2 : using function approximation. 
    # Y_col = "modalPrice"
    # X_col = ["minPrice"]
    ################################################################
    df_yes_na = df[df[Y_col].isna()]
    if(len(df_yes_na) == 0):
        print("Nothing to predict, no null values!!!!")
        return df
    
    ################################################################
    # One without no null values in modalprice 
    #Filtering data 
    df_not_na = df[df[Y_col].notna()]
    for x_col_name in X_col:
        df_not_na = df_not_na[df_not_na[x_col_name].notna()]
    # Defining a linear model for mapping x = ["minPrice"], y = "modalPrice"
    X = np.array(np.matrix(df_not_na[X_col]))
    Y = np.array(np.matrix(df_not_na[Y_col]).T) 
    #print(X)
    try: 
        lin_reg   = LinearRegression()
        lin_model = lin_reg.fit(X,Y)
    except:
        print("The Linear regression model didn't run!!!!. Problem with the data. ")
    ################################################################
    ##Replacing the predicted values!!!!....
    X_test    = np.array(np.matrix(df_yes_na[X_col]))
    Y_test    = lin_model.predict(X_test)
    list_index_mis_values = list(df_yes_na.index)
    df.loc[list_index_mis_values,Y_col] = Y_test
    ################################################################
    
    ##testing if null values are replaces using linear regression method. 
    try:
        assert len(df[df[Y_col].isna()])== 0
    except:
        print("The null values are not replaced correctly!!!. Don't execute next code.")
    
    ################################################################
    
    return df





def handle_outlier(df,col_name):
    # Outlier Handling. 
    # Going to detect outlier based on standard deviation. if the data point is mean + (3* Std) --> Then classify as outlier
    # treat the point as mean
    #col_name = "mis_m2_minprice"
    mean = df[col_name].mean()
    std  = df[col_name].std()
    out_pos = mean + (3 * std)
    out_neg = mean - (3 * std)
    cond = (df[col_name] > out_pos) | (df[col_name] < out_neg)
    if(len(df[cond])==0):
        print("No outliers to Handle!!!!!!!")
        return df
    else:
        df.loc[cond,col_name] = mean
        return df 

In [4]:
df = pd.read_csv("azd.csv")

In [5]:
df.head()

,priceDate,itemName,state,mandiName,arrivals,unitArrivals,variety,minPrice,maxPrice,modalPrice,priceUnit
0,2005-04-11,Potato,NCT of Delhi,Azadpur,106.0,Tonnes,F.A.Q.,300.0,663.0,475.0,Rs/Quintal
1,2005-04-12,Potato,NCT of Delhi,Azadpur,1745.0,Tonnes,F.A.Q.,325.0,688.0,500.0,Rs/Quintal
2,2005-04-13,Potato,NCT of Delhi,Azadpur,1233.0,Tonnes,F.A.Q.,300.0,688.0,488.0,Rs/Quintal
3,2005-04-16,Potato,NCT of Delhi,Azadpur,1654.0,Tonnes,F.A.Q.,350.0,650.0,475.0,Rs/Quintal
4,2005-04-18,Potato,NCT of Delhi,Azadpur,26.0,Tonnes,F.A.Q.,350.0,650.0,475.0,Rs/Quintal


In [6]:
#Filtering the data. 
df=df[['priceDate','arrivals','minPrice','maxPrice','modalPrice']] 

In [7]:
df["mis_m1_minprice"]    = df["minPrice"]
df["mis_m1_maxprice"]    = df["maxPrice"]
df["mis_m1_modalprice"]  = df["modalPrice"]
####
df["mis_m2_minprice"]    = df["minPrice"]
df["mis_m2_maxprice"]    = df["maxPrice"]
df["mis_m2_modalprice"]  = df["modalPrice"]
####

In [8]:
#Writing Descriptive analysis. 
file_path ="csv_output/descriptive_analysis.csv"
if os.path.exists(file_path):
    print("File already written!!!.....")
else:
    df.describe().to_csv(file_path)

File already written!!!.....


In [9]:
# Missing Value Treatment
#Method 1 : Using mean to substitute missing values. 
col_name = "mis_m1_minprice"
df = replace_mis_val_mean(df,col_name)

col_name = "mis_m1_maxprice"
df = replace_mis_val_mean(df,col_name)


col_name = "mis_m1_modalprice"
df = replace_mis_val_mean(df,col_name)




In [10]:
# Missing Value Treatment
#Method 2 : Using Linear Regression for filling missing values 
col_name = "mis_m2_minprice"
df = mis_replace_linear_regression(df,X_col= ["maxPrice","modalPrice"],Y_col=col_name)

col_name = "mis_m2_maxprice"
df = mis_replace_linear_regression(df,X_col=["minPrice"] ,Y_col=col_name)


col_name = "mis_m2_modalprice"
df = mis_replace_linear_regression(df,X_col=["minPrice"] ,Y_col=col_name)


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [55]:
col_name ="mis_m2_minprice"
df = handle_outlier(df,col_name)

col_name = "mis_m2_maxprice"
df = handle_outlier(df,col_name)

col_name = "mis_m2_modalprice"
df = handle_outlier(df,col_name)

No outliers to Handle!!!!!!!


In [15]:
df_new = df.copy()

In [19]:
df_ts =df_new[["priceDate","mis_m2_minprice","mis_m2_maxprice","mis_m2_modalprice"]]

In [24]:
df_ts.columns = ["timestamp","min_price","max_price","modal_price"]

In [ ]:
df.priceDate = pd.to_datetime(df.priceDate) #conveting the date column as index

In [ ]:
df.set_index('priceDate', inplace=True)

In [ ]:
df.plot()
plt.show()

In [ ]:
df.plot(use_index=True,y='minPrice')

In [ ]:
#mean = numpy.mean(elements, axis=0)
#sd = numpy.std(elements, axis=0)
#df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
#def remove_outlier(df, col_name):
 #   q1 = df[col_name].quantile(0.25)
 #   q3 = df[col_name].quantile(0.75)
  #  iqr = q3-q1 #Interquartile range
   # fence_low  = q1-1.5*iqr
    #fence_high = q3+1.5*iqr
    #df_out = df.loc[(df[col_name] > fence_low) & (df[col_name] < fence_high)]
    #return df_out
#final_list = [x for x in df['maxPrice'] if (x > mean - 2 * sd)]
#final_list = [x for x in final_list if (x < mean + 2 * sd)]

In [ ]:
# remove_outlier(df,'maxPrice')
# remove_outlier(df,'modalPrice')
# remove_outlier(df,'minPrice')

In [ ]:
df.describe()

In [ ]:
df.plot(use_index=True,y='modalPrice')

In [ ]:
df.plot(use_index=True,y='maxPrice')

In [ ]:
df.plot(use_index=True,y='modalPrice')

In [ ]:
#rolmean=df.rolling(window=3).mean()
def add_mean_std_cols(df,col_name,t):
    res = df[col_name].rolling(t).agg(['mean'])
    #res.columns = res.columns.map('_'.join)
    #cols = np.concatenate(list(zip(df.columns, res.columns[0::2], res.columns[1::2])))
    #final = res.join(df).loc[:, cols]
    return res


#for i in range(2,10):
#   df['MA{}'.format(i)] = df.rolling(window=i).mean()


In [ ]:
n_df_min=add_mean_std_cols(df,["minPrice"],3)
n_df_max=add_mean_std_cols(df,["maxPrice"],3)
n_df_mode=add_mean_std_cols(df,["modalPrice"],3)

In [ ]:
#original=plt.plot(df,color='gray',label='original')
mean=plt.plot(n_df_min,label='mean')

In [ ]:
#original=plt.plot(df,color='gray',label='original')
mean_max=plt.plot(n_df_max,label='mean')

In [ ]:
#original=plt.plot(df,color='gray',label='original')
mean_modal=plt.plot(n_df_mode,label='mean')

In [ ]:
from 